Probability Calibration for KG Embedding in Entity typing task
==========

Test various calibration techniques for TransE, ComplEx, HoLE, DistMult on two datasets includeing entity type information: [YAGO-ET and DBpedia-ET](https://github.com/JunhengH/joie-kdd19/tree/master/data).

We check if 
1. the calibration techqniues work;
2. using calibrated probability and a natural threshold 0.5 could achieve STOA as fine-tuned scores

In [1]:
import sys
# enable importing the modules from probcalkge
sys.path.append('../')
sys.path.append('../probcalkge')

In [2]:
from pprint import pprint
from typing import Iterable, Callable, Union
import numpy as np
import pandas as pd

In [3]:
from probcalkge import Experiment, ExperimentResult
from probcalkge import get_calibrators
from probcalkge import get_datasets,  get_kgemodels
from probcalkge import brier_score, negative_log_loss, ks_error, ece

In [4]:
ds = get_datasets()
# from probcalkge.calmodels import get_calibrators
from probcalkge.calmodels2 import get_calibrators
cals = get_calibrators()
kges = get_kgemodels()


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


In [5]:
from probcalkge.caldatasets import get_yago_et
yagoet = get_yago_et()

In [6]:
from ampligraph.latent_features import DistMult
from ampligraph.evaluation import evaluate_performance, mrr_score, hits_at_n_score
fil = np.concatenate((yagoet.X_train, yagoet.X_valid, yagoet.X_test))
# dist = DistMult(verbose=True, batches_count=10, seed=0, epochs=20, k=150, eta=10,)
# dist.fit(yagoet.X_train)
transE = kges.transE
transE.fit(yagoet.X_train)
# et_test = [trp for trp in yagoet.X_test if trp[1]=='type']
ranks = evaluate_performance(yagoet.X_test, 
                                model=transE, filter_triples=fil)

100%|██████████| 49268/49268 [37:48<00:00, 21.72it/s]


In [7]:
# compute and print metrics:
mrr = mrr_score(ranks)
hits_10 = hits_at_n_score(ranks, n=10)
print("MRR: %f, Hits@10: %f" % (mrr, hits_10))

MRR: 0.049808, Hits@10: 0.077728


In [8]:
exp = Experiment(
    cals=[cals.uncal, cals.platt, cals.isot, cals.histbin, cals.beta], 
    datasets=[ds.yago_et], 
    kges=[ kges.distMult], 
    metrics=[brier_score, negative_log_loss, ks_error, ece]
    )

exp.train_kges()
exp.save_trained_kges('../saved_kges')
# exp.load_trained_kges('../saved_models/')
exp_res = exp.run_with_trained_kges()

exp_res.to_frame().to_csv('res.csv')

training DistMult on YAGO_ET ...


Average DistMult Loss:   0.814482:  58%|█████▊    | 58/100 [21:56<15:53, 22.69s/epoch]


KeyboardInterrupt: 

In [9]:
exp.save_trained_kges('../saved_models/')

AttributeError: 'NoneType' object has no attribute 'all_params'